In [ ]:
# https://pjoshi15.com/merging-multiple-lora-models-python/
import os
import subprocess
if not os.path.isfile("../../cache/papercut.safetensors"):
    subprocess.check_call(cwd="../../cache", ["curl", "-sSLOJ", "https://civitai.com/api/download/models/133503"])
    print("Downloaded papercut")
if not os.path.isfile("Aether_Beauty_v1_SDXL_LoRA.safetensors"):
    subprocess.check_call(cwd="../../cache", ["curl", "-sSLOJ", "https://civitai.com/api/download/models/335725?type=Model&format=SafeTensor"])
    print("Downloaded Aether_Beauty_v1_SDXL_LoRA.safetensors")

In [ ]:
import time

import accelerate
import compel
import diffusers
import torch
from IPython.display import display

DEVICE = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
DTYPE = torch.float16 if DEVICE in ("cuda", "mps") else torch.float32
print(f"Using {DEVICE}")

def get_generator(seed):
    if DEVICE in ("cuda", "mps"):
        return torch.Generator(DEVICE).manual_seed(seed)
    return torch.Generator().manual_seed(seed)

with accelerate.init_empty_weights():
    pipe = diffusers.DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", variant="fp16")
pipe.load_lora_weights("latent-consistency/lcm-lora-sdxl")
pipe.scheduler = diffusers.LCMScheduler.from_config(pipe.scheduler.config)
_ = pipe.to(DEVICE, dtype=DTYPE)

In [ ]:
image = pipe(
    "hiker, beautiful forest",
    num_inference_steps=4, guidance_scale=1,
    generator=get_generator(0)).images[0]
image

In [ ]:
pipe.load_lora_weights(pretrained_model_name_or_path_or_dict="../../cache/papercut.safetensors", adapter_name="papercut")

# Regulate contribution of LoRA model in image generation.
lora_scale = 0.9

image = pipe(
    "papercut, hiker, beautiful forest",
    num_inference_steps=4, guidance_scale=1,
    cross_attention_kwargs={"scale": lora_scale}, 
    generator=get_generator(0)).images[0]

image

In [ ]:
# https://civitai.com/models/298913/aether-beauty-lora-for-sdxl
# Fuse the papier one
pipe.fuse_lora(lora_scale = 0.7)

pipe.load_lora_weights(pretrained_model_name_or_path_or_dict="../../cache/Aether_Beauty_v1_SDXL_LoRA.safetensors", adapter_name="aether")
pipe.fuse_lora(lora_scale = 0.7)

pipe.set_adapters(["papercut", "aether"], adapter_weights=[1.0, 0.5])

In [ ]:

image = pipe(
    prompt="papercut, big eyes, big teeth, photo, hiker, beautiful forest",
    num_inference_steps=4, guidance_scale=1,
    cross_attention_kwargs={"scale": lora_scale}, 
    generator=get_generator(0)).images[0]

image